In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format
random_state = 42
from datetime import date
import plotly.graph_objects as go
from plotly import offline as pyoff



from datetime import datetime, date, time, timedelta
import calendar

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import time
from xgboost import plot_importance
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb


In [3]:
###Vamos a hacer una estimacion del exito de la campaña. Como no tenemos un historico de campañas, esstimamos que un 30% de 
##de los clientes, pueden llegar a leerse el email

In [4]:
df = pd.read_csv('10000clientes.csv')

In [5]:
df

,pk_cid,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,region_code,salary,age,gender
0,1517456,161,6,910.00,1,4,1,141,28.00,"113,652.32",37.00,2.00
1,1512063,178,6,900.00,1,4,1,141,8.00,"113,652.32",32.83,2.00
2,1515010,169,6,900.00,1,4,1,141,28.00,"113,652.32",31.00,1.00
3,1520203,148,6,840.00,1,4,1,141,8.00,"113,652.32",34.50,1.00
4,1518210,157,6,810.00,1,4,1,141,28.00,"113,652.32",65.00,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...
9967,1401318,351,12,640.00,4,1,1,411,28.00,"152,395.11",31.33,1.00
9968,1397743,374,13,640.00,4,1,1,411,35.00,"88,566.72",27.08,1.00
9969,1397057,379,12,640.00,4,1,1,411,28.00,"84,545.25",45.00,2.00
9970,1399722,361,12,640.00,4,1,1,411,33.00,"46,825.86",42.67,2.00


In [6]:
df["salary"] = df["salary"].astype(int)

df["age"] = df["age"].astype(int)

df["gender"] = df["gender"].astype(int)

df["region_code"] = df["region_code"].astype(int)
df["monetary_value"] = df["monetary_value"].astype(int)

In [7]:
df= df.sample(n=3000, random_state=42)

In [8]:
##Lo contemplamos como un modelo supervisado por clasificacion, porque queremos saber, de esa estimacion del 30% de clientes
##que van a tener interes despues de recibir el email, que porcentaje, va a comprar o no, y por tanto ,generara ingresos, o no.

In [9]:
###Para este dataset, vamos a predecir el incremento de revenue, que puede haber

In [10]:
y= df['monetary_value']

In [11]:
X= df

In [12]:
X= X.drop('monetary_value', axis=1)

In [13]:
from sklearn.model_selection import train_test_split
random_seed = 42
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)
                                                    


In [14]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer as mlb


In [15]:
from lightgbm import LGBMModel,LGBMClassifier

In [16]:
t0 = time.time()
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train,y_train)
y_pred = lgbm_model.predict(X_test)
proba = lgbm_model.predict_proba(X_test)

cv_score = cross_val_score(lgbm_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
lgbm_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.11212121212121212
CV Score : 0.10597014925373135
              precision    recall  f1-score   support

         120       0.14      0.12      0.13        24
         130       0.06      0.17      0.09         6
         140       0.28      0.36      0.31        14
         150       0.28      0.28      0.28        32
         160       0.21      0.24      0.23        25
         170       0.08      0.06      0.07        17
         180       0.09      0.08      0.08        13
         190       0.00      0.00      0.00        22
         200       0.12      0.12      0.12         8
         210       0.00      0.00      0.00        18
         220       0.26      0.46      0.33        26
         230       0.08      0.16      0.10        19
         240       0.10      0.07      0.08        15
         250       0.00      0.00      0.00        11
         260       0.00      0.00      0.00         6
         270       0.00      0.00      0.00         6
         280       

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [17]:
t0 = time.time()
random_model = RandomForestClassifier()
random_model.fit(X_train,y_train)
y_pred = random_model.predict(X_test)
proba = random_model.predict_proba(X_test)

cv_score = cross_val_score(random_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
random_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.10202020202020202
CV Score : 0.10398009950248757
              precision    recall  f1-score   support

         120       0.04      0.04      0.04        24
         130       0.13      0.33      0.19         6
         140       0.21      0.36      0.26        14
         150       0.28      0.31      0.29        32
         160       0.24      0.20      0.22        25
         170       0.00      0.00      0.00        17
         180       0.00      0.00      0.00        13
         190       0.00      0.00      0.00        22
         200       0.12      0.12      0.12         8
         210       0.17      0.06      0.08        18
         220       0.23      0.46      0.30        26
         230       0.15      0.26      0.19        19
         240       0.20      0.13      0.16        15
         250       0.00      0.00      0.00        11
         260       0.17      0.17      0.17         6
         270       0.00      0.00      0.00         6
         280       

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
t0 = time.time()
decission_model = DecisionTreeClassifier()
decission_model.fit(X_train,y_train)
y_pred =decission_model.predict(X_test)
proba =decission_model.predict_proba(X_test)

cv_score = cross_val_score(decission_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
decission_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.08383838383838384
CV Score : 0.07213930348258707
              precision    recall  f1-score   support

         120       0.07      0.04      0.05        24
         130       0.14      0.33      0.20         6
         140       0.19      0.29      0.23        14
         150       0.24      0.28      0.26        32
         160       0.21      0.20      0.20        25
         170       0.09      0.06      0.07        17
         180       0.12      0.15      0.13        13
         190       0.12      0.05      0.07        22
         200       0.13      0.25      0.17         8
         210       0.20      0.17      0.18        18
         220       0.22      0.27      0.24        26
         230       0.10      0.11      0.10        19
         240       0.00      0.00      0.00        15
         250       0.00      0.00      0.00        11
         260       0.00      0.00      0.00         6
         270       0.00      0.00      0.00         6
         280       

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [19]:
t0 = time.time()
xgbc_model = DecisionTreeClassifier()
xgbc_model.fit(X_train,y_train)
y_pred =xgbc_model.predict(X_test)
proba =xgbc_model.predict_proba(X_test)

cv_score = cross_val_score(xgbc_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
xgbc_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.08484848484848485
CV Score : 0.07412935323383084
              precision    recall  f1-score   support

         120       0.05      0.04      0.05        24
         130       0.14      0.33      0.20         6
         140       0.24      0.29      0.26        14
         150       0.24      0.28      0.26        32
         160       0.29      0.24      0.26        25
         170       0.20      0.12      0.15        17
         180       0.07      0.08      0.07        13
         190       0.00      0.00      0.00        22
         200       0.07      0.12      0.09         8
         210       0.12      0.11      0.12        18
         220       0.24      0.31      0.27        26
         230       0.10      0.11      0.10        19
         240       0.00      0.00      0.00        15
         250       0.00      0.00      0.00        11
         260       0.00      0.00      0.00         6
         270       0.00      0.00      0.00         6
         280       

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [20]:
##El mejor modelo es el LGBMClassifier, que pronóstica, que alrededor del 11% de clientes, interesados, acabaran comprando.

In [21]:
##SI establecemos que la campaña, tiene una duración de 2 meses, vamos a mirar, lo que han facturado los clientes, en 
##estos 2 ultimos meses

In [22]:
df= df.sample(n=3000, random_state=42)

In [23]:
df

,pk_cid,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,region_code,salary,age,gender
8921,1418664,305,11,710,4,1,1,411,3,45140,40,1
1785,1515915,165,6,260,1,4,1,141,11,113652,47,2
3973,1541920,60,3,140,1,4,1,141,30,113652,20,1
4417,1524642,133,5,120,1,4,1,141,23,113652,44,2
6060,1377511,492,17,1060,4,1,1,411,36,53210,42,2
...,...,...,...,...,...,...,...,...,...,...,...,...
7293,1389355,429,15,870,4,1,1,411,41,86454,37,1
4423,123603,123,5,120,1,4,1,141,28,113652,61,2
6523,1387983,438,15,990,4,1,1,411,28,113652,43,1
8167,1402267,344,12,780,4,1,1,411,26,118618,38,1


In [24]:
df['recency']<=90

8921    False
1785    False
3973     True
4417    False
6060    False
        ...  
7293    False
4423    False
6523    False
8167    False
50      False
Name: recency, Length: 3000, dtype: bool

In [25]:
df_ventas = df.groupby(by=['pk_cid','recency'],as_index=False)["monetary_value"].sum()

In [26]:
df_ventas

,pk_cid,recency,monetary_value
0,32376,483,1740
1,39008,134,210
2,48604,484,700
3,61578,457,900
4,64859,400,800
...,...,...,...
2995,1545940,40,140
2996,1546993,36,130
2997,1547343,33,120
2998,1549724,19,120


In [27]:
df_ventas['recency']<=90

0       False
1       False
2       False
3       False
4       False
        ...  
2995     True
2996     True
2997     True
2998     True
2999     True
Name: recency, Length: 3000, dtype: bool

In [28]:
df_ventas_prediccion = df_ventas[df_ventas['recency'] <= 90]


In [30]:
df_ventas_prediccion['monetary_value'].sum()

31450

In [36]:
##Los ingresos previstos, son de 3437,5€, teniendo en cuenta la prevision de que venderemos un 11%

In [37]:
###Habiendo mirado que el coste que tenemos por enviar un email a 10000 clientes ,es de 100€, el ROI es de un 33,27%